In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Polycystic_Kidney_Disease"
cohort = "GSE74451"

# Input paths
in_trait_dir = "../../input/GEO/Polycystic_Kidney_Disease"
in_cohort_dir = "../../input/GEO/Polycystic_Kidney_Disease/GSE74451"

# Output paths
out_data_file = "../../output/preprocess/Polycystic_Kidney_Disease/GSE74451.csv"
out_gene_data_file = "../../output/preprocess/Polycystic_Kidney_Disease/gene_data/GSE74451.csv"
out_clinical_data_file = "../../output/preprocess/Polycystic_Kidney_Disease/clinical_data/GSE74451.csv"
json_path = "../../output/preprocess/Polycystic_Kidney_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of a novel risk factor for intracranial aneurysms in ADPKD using iPSC models [Agilent]"
!Series_summary	"Cardiovascular complications are the leading cause of death in autosomal dominant polycystic kidney disease (ADPKD), and intracranial aneurysm (ICA) causing subarachnoid hemorrhage is among the most serious complications. The diagnostic and therapeutic strategies for ICAs in ADPKD have not been fully established. We here generated induced pluripotent stem cells (iPSCs) from seven ADPKD patients, including four with ICAs. The vascular cells differentiated from ADPKD-iPSCs showed altered Ca2+ entry and gene expression profiles compared with those from control-iPSCs. We found that the expression level of a metalloenzyme gene, matrix metalloproteinase (MMP) 1, was specifically elevated in the iPSC-derived endothelia from ADPKD patients with ICAs. Furthermore, we confirmed a statistically significant correlation between the serum MMP1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any
import json

# 1. Gene Expression Data Availability
# From the background info, this dataset appears to contain gene expression data from iPSC-derived cells
is_gene_available = True

# 2.1 Data Availability
# Trait (ADPKD status) is at index 1 in the Sample Characteristics Dictionary
trait_row = 1

# Age is not available in the sample characteristics dictionary
age_row = None

# Gender is at index 2 and 4 (duplicate) in the Sample Characteristics Dictionary
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert ADPKD trait status to binary (0 = control, 1 = ADPKD)"""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
        if 'adpkd' in value or 'polycystic kidney disease' in value:
            return 1
        elif 'control' in value or 'healthy' in value:
            return 0
    return None

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)"""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
        if 'male' in value:
            return 1
        elif 'female' in value:
            return 0
    return None

# Age conversion function (not used as age data is not available)
def convert_age(value):
    """Convert age to continuous value"""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
        try:
            return float(value)
        except ValueError:
            pass
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is already available from previous steps
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Selected Clinical Data Preview:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Selected Clinical Data Preview:
{'GSM1920920': [0.0, 1.0], 'GSM1920921': [0.0, 1.0], 'GSM1920922': [0.0, 1.0], 'GSM1920923': [0.0, 1.0], 'GSM1920924': [0.0, 1.0], 'GSM1920925': [0.0, 1.0], 'GSM1920926': [0.0, 1.0], 'GSM1920927': [0.0, 1.0], 'GSM1920928': [0.0, 1.0], 'GSM1920929': [0.0, 1.0], 'GSM1920930': [0.0, 1.0], 'GSM1920931': [0.0, 1.0], 'GSM1920932': [0.0, 1.0], 'GSM1920933': [0.0, 1.0], 'GSM1920934': [0.0, 1.0], 'GSM1920935': [0.0, 1.0], 'GSM1920936': [0.0, 1.0], 'GSM1920937': [1.0, 1.0], 'GSM1920938': [1.0, 1.0], 'GSM1920939': [1.0, 1.0], 'GSM1920940': [1.0, 1.0], 'GSM1920941': [1.0, 1.0], 'GSM1920942': [1.0, 1.0], 'GSM1920943': [1.0, 1.0], 'GSM1920944': [1.0, 1.0], 'GSM1920945': [1.0, 1.0], 'GSM1920946': [1.0, 1.0], 'GSM1920947': [1.0, 1.0], 'GSM1920948': [1.0, 1.0], 'GSM1920949': [1.0, 1.0], 'GSM1920950': [1.0, 1.0]}
Clinical data saved to ../../output/preprocess/Polycystic_Kidney_Disease/clinical_data/GSE74451.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These gene identifiers appear to be probe IDs from a microarray platform, not standard human gene symbols
# The identifiers have formats like "(+)E1A_r60_1" and "A_19_P00315452" which are not standard gene symbols
# They need to be mapped to proper human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns in gene_annotation containing gene identifiers and gene symbols
# From the preview, 'ID' contains probe identifiers (e.g., A_23_P117082) matching the gene expression data
# 'GENE_SYMBOL' contains the human gene symbols (e.g., HEBP1, KCNE4)
id_column = 'ID'
gene_symbol_column = 'GENE_SYMBOL'

# 2. Create gene mapping dataframe by extracting the two relevant columns
gene_mapping = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows of gene expression data after mapping:")
print(gene_data.head())

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Check for some common gene symbols to verify mapping worked properly
common_genes = ['TP53', 'BRCA1', 'KRAS', 'TNF']
found_genes = [gene for gene in common_genes if gene in gene_data.index]
print(f"Found {len(found_genes)} common genes out of {len(common_genes)} checked: {found_genes}")


Gene mapping dataframe shape: (46204, 2)
First few rows of gene mapping:
              ID          Gene
2   A_23_P117082         HEBP1
3  A_33_P3246448         KCNE4
4  A_33_P3318220        BPIFA3
5  A_33_P3236322  LOC100129869
6  A_33_P3319925          IRG1
Gene expression data shape after mapping: (20353, 31)
First few rows of gene expression data after mapping:
          GSM1920920  GSM1920921  GSM1920922  GSM1920923  GSM1920924  \
Gene                                                                   
A1BG        1.555157    0.039984   -0.678945   -2.468467   -0.687164   
A1BG-AS1   -0.031899    0.399478    0.491183   -0.043165    0.327540   
A1CF       -7.052572   -6.938511   -6.495134   -6.282820   -6.263671   
A2LD1      -2.554118   -2.768966   -1.938280   -2.780313   -0.552746   
A2M         2.396515    3.753389    2.026154    2.604408    3.323712   

          GSM1920925  GSM1920926  GSM1920927  GSM1920928  GSM1920929  ...  \
Gene                                               

Gene expression data saved to ../../output/preprocess/Polycystic_Kidney_Disease/gene_data/GSE74451.csv
Found 4 common genes out of 4 checked: ['TP53', 'BRCA1', 'KRAS', 'TNF']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was processed in step 2
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# The clinical dataframe has samples as columns. Need to properly transpose and name columns
clinical_df_t = clinical_df.set_index(clinical_df.columns[0]).T
# Row 0 corresponds to the trait (Polycystic_Kidney_Disease) and row 1 corresponds to Gender
clinical_df_t.columns = [trait, 'Gender']
print(f"Transposed clinical data shape: {clinical_df_t.shape}")
print(clinical_df_t.head())

# 3. Link the clinical and genetic data
linked_data = pd.concat([clinical_df_t, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.iloc[:5, :5])  # Print first 5 rows and 5 columns

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from iPSC-derived endothelial and smooth muscle cells from ADPKD patients and controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19847, 31)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']


Normalized gene data saved to ../../output/preprocess/Polycystic_Kidney_Disease/gene_data/GSE74451.csv
Loaded clinical data shape: (2, 31)
   GSM1920920  GSM1920921  GSM1920922  GSM1920923  GSM1920924  GSM1920925  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         1.0         1.0         1.0         1.0         1.0         1.0   

   GSM1920926  GSM1920927  GSM1920928  GSM1920929  ...  GSM1920941  \
0         0.0         0.0         0.0         0.0  ...         1.0   
1         1.0         1.0         1.0         1.0  ...         1.0   

   GSM1920942  GSM1920943  GSM1920944  GSM1920945  GSM1920946  GSM1920947  \
0         1.0         1.0         1.0         1.0         1.0         1.0   
1         1.0         1.0         1.0         1.0         1.0         1.0   

   GSM1920948  GSM1920949  GSM1920950  
0         1.0         1.0         1.0  
1         1.0         1.0         1.0  

[2 rows x 31 columns]
Transposed clinical data shape: (30, 2)
   

Shape after handling missing values: (30, 19849)
For the feature 'Polycystic_Kidney_Disease', the least common label is '1.0' with 14 occurrences. This represents 46.67% of the dataset.
The distribution of the feature 'Polycystic_Kidney_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 30 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



A new JSON file was created at: ../../output/preprocess/Polycystic_Kidney_Disease/cohort_info.json


Linked data saved to ../../output/preprocess/Polycystic_Kidney_Disease/GSE74451.csv
